<a href="https://colab.research.google.com/github/ttogle918/news_by_kobert/blob/master/model_epoch5_kobert_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
from google.colab import drive

drive.mount('/content/drive')
data_path = '/content/drive/My Drive/Colab Notebooks/news_class9x1400'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers
!pip install torch

In [3]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-xdyfrwbn
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-xdyfrwbn


In [4]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [5]:
# gpu 연산이 가능하면 'cuda:0', 아니면 'cpu' 출력
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [6]:
#bert 모델, vocab 불러오기
bertmodel, vocab = get_pytorch_kobert_model()

using cached model. /content/.cache/kobert_v1.zip
using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [29]:
import os
max_seq_len = 128
dataset_train = []
labels = []
for (path, dir, files) in os.walk(data_path):
    for filename in files:
        ext = os.path.splitext(filename)[-1]
        if ext == '.txt':
            with open("%s/%s" % (path, filename), encoding="utf-8") as f:
              # labels.append( path[path.rindex('/')+1:])
              # dataset_train.append(f.read()[:max_seq_len])
              
              label = path[path.rindex('/')+1:]
              text = f.read()
              text = text.split('\n')
              saved_text, s, i = '', 0, 0
              for t in text :
                  if i == 2 :
                      break
                  if len(t) > max_seq_len :
                      labels.append( label )
                      dataset_train.append(saved_text)
                      labels.append( label )
                      dataset_train.append(t)
                      saved_text, s = '', 0
                      i += 1
                  elif s + len(t) > max_seq_len :
                      labels.append( label )
                      dataset_train.append(saved_text)
                      saved_text, s = t, len(t)
                      i += 1
                  else :
                      saved_text += t
                      s += len(t)
              labels.append( label )
              dataset_train.append(saved_text)


len(dataset_train)

41118

## 전처리

In [30]:
import pandas as pd

# labels_classes
df = pd.DataFrame({'content' : dataset_train, 'label' : labels})

# dataset의 balance 여부 확인
df.groupby(by=['label']).count()

,content
label,
ITscience,4504
culture,4543
economy,4618
entertainment,4544
health,4473
life,4517
politic,4771
social,4622
sport,4526


In [31]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
label_encoder.fit(df['label'])
num_labels = len(label_encoder.classes_)

df['encoded_label'] = np.asarray(label_encoder.transform(df['label']), dtype=np.int32)
df.head(3)

,content,label,encoded_label
0,"원칙 있는 사회를 갈망한 노무현을 만나다노무현 전 대통령의 서거 후, 얼마 지나지 ...",social,7
1,"전 대통령이 스스로 목숨을 끊은 것은 무책임이라는 교수의 말에 내가 그만, 발끈해 ...",social,7
2,무책임이라는 말은 신념을 위해 자신의 목숨을 던진 정치가에 대한 평가로 적절치 않다...,social,7


In [32]:
from sklearn.model_selection import train_test_split

# Split Train and Validation data
train_texts, test_texts, train_labels, test_labels = train_test_split(df["content"].to_list(), df["encoded_label"].to_list(), test_size=0.2, random_state=0, shuffle=True, stratify=df["encoded_label"].to_list())

print(len(train_texts), len(train_labels), len(test_texts))

32894 32894 8224


In [33]:
dataset_train, dataset_test = [], []

for text, label in zip(train_texts, train_labels) :
  dataset_train.append([text, label])
for text, label in zip(test_texts, test_labels) :
  dataset_test.append([text, label])
len(dataset_train), len(dataset_test)

(32894, 8224)

In [34]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [35]:
## Setting parameters
max_len = 128
batch_size = 32
warmup_ratio = 0.1
num_epochs = 10
max_grad_norm = 1
log_interval = 200

In [36]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)
#BERTDataset 클래스 이용, TensorDataset으로 만들어주기
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [37]:
#배치 및 데이터로더 설정
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=4)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=4)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [38]:
del df
# del data_train
# del data_test
# del tok
# del tokenizer
# del dataset_train
# del dataset_test
del train_texts
del train_labels

In [39]:
class BERTClassifier(nn.Module):
    def __init__(self, bert, hidden_size = 768,
                 num_classes=num_labels, ## 분류할 클래스 수
                 dr_rate=None, params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [40]:
model = BERTClassifier(bertmodel, dr_rate=0.5).to(device) #gpu

In [41]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [42]:
def train_for_epochs(num_epochs, train_dataloader, test_dataloader) :
  # Prepare optimizer and schedule (linear warmup and decay)
  no_decay = ['bias', 'LayerNorm.weight']
  optimizer_grouped_parameters = [
      {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
      {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
  ]

  optimizer = AdamW(optimizer_grouped_parameters, lr=5e-5)  # lr = learning rate
  loss_fn = nn.CrossEntropyLoss()

  t_total = len(train_dataloader) * num_epochs
  warmup_step = int(t_total * warmup_ratio)

  scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)
  
  # train
  for e in range(num_epochs):
      train_acc = 0.0
      test_acc = 0.0
      # training
      model.train()
      for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
          optimizer.zero_grad()
          token_ids = token_ids.long().to(device)
          segment_ids = segment_ids.long().to(device)
          valid_length= valid_length
          label = label.long().to(device)
          out = model(token_ids, valid_length, segment_ids)
          loss = loss_fn(out, label)
          loss.backward()
          torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
          optimizer.step()
          scheduler.step()  # Update learning rate schedule
          train_acc += calc_accuracy(out, label)
          if batch_id % log_interval == 0:
              print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
      print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
      model.eval()
      # validation
      for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
          token_ids = token_ids.long().to(device)
          segment_ids = segment_ids.long().to(device)
          valid_length= valid_length
          label = label.long().to(device)
          out = model(token_ids, valid_length, segment_ids)
          test_acc += calc_accuracy(out, label)
      print("epoch {} validation acc {}".format(e+1, test_acc / (batch_id+1)))
      torch.save({
            'epoch': epoch,
            'optimizer_state_dict': optimizer.state_dict(),
            'model_state_dict': model.state_dict(),
            'input_dim': args.input_dim
        }, f'{args.output_prefix}.{epoch:04d}.pth')

## 1.

+ 결과

  + epoch 5 train acc 0.977254746835443
  + epoch 5 validation acc 0.8131510416666666
+ Setting parameters
    + max_len = 64
    + batch_size = 64
    + warmup_ratio = 0.1
    + num_epochs = 5
    + max_grad_norm = 1
    + log_interval = 200
    + learning_rate =  5e-5


In [21]:
# train_for_epochs(num_epochs, train_dataloader, test_dataloader)

## 2

+ 결과
  + epoch 5 train acc 0.9723214285714286
  + epoch 5 validation acc 0.8468253968253968
+ 파라미터
  + max_len = 128
  + batch_size = 32
  + warmup_ratio = 0.1
  + num_epochs = 5
  + max_grad_norm = 1
  + log_interval = 200
  + learning_rate =  5e-5

## 3.

+ 결과
  + epoch 5 train acc 0.9745039682539682
  + epoch 5 validation acc 0.8631329113924051
+ 파라미터
  + max_len = 256
  + batch_size = 32
  + warmup_ratio = 0.1
  + num_epochs = 5
  + max_grad_norm = 1
  + log_interval = 200
  + learning_rate =  5e-5

## 4.

+ 결과
  + epoch 5 train acc 0.9691468253968254
  + epoch 5 validation acc 0.8753968253968254
+ 파라미터
  + max_len = 512
  + batch_size = 8
  + warmup_ratio = 0.1
  + num_epochs = 5
  + max_grad_norm = 1
  + log_interval = 200

In [43]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [26]:
train_for_epochs(5, train_dataloader, test_dataloader)

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/315 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epoch 1 batch id 1 loss 0.005040772724896669 train acc 1.0
epoch 1 batch id 201 loss 0.247959703207016 train acc 0.9821206467661692
epoch 1 train acc 0.973313492063492


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:41: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


...

epoch 4 validation acc 0.8231803797468354


  0%|          | 0/315 [00:00<?, ?it/s]

epoch 5 batch id 1 loss 0.001841771649196744 train acc 1.0
epoch 5 batch id 201 loss 0.0015497547574341297 train acc 0.9964241293532339
epoch 5 train acc 0.9963293650793651


  0%|          | 0/79 [00:00<?, ?it/s]

epoch 5 validation acc 0.8275316455696202


In [27]:
torch.save(model, data_path+'model_epoch5.pt')